In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree
import os

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
csv_path = os.path.join(project_root, "Data", "xp.csv")
csv_path1 = os.path.join(project_root, "Data", "metro.csv")

df = pd.read_csv(csv_path)

metro_df = pd.read_csv(csv_path1)

metro_df.columns = metro_df.columns.str.strip()

In [3]:
df['date_mutation'] = pd.to_datetime(df['date_mutation'])

df['year'] = df['date_mutation'].dt.year
df['month'] = df['date_mutation'].dt.month
df['day_of_week'] = df['date_mutation'].dt.dayofweek
df['days_since_start'] = (df['date_mutation'] - df['date_mutation'].min()).dt.days

In [4]:
#Clusters
geo_features = df[['longitude', 'latitude']].copy()
n_clusters = 20

km = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df['geo_cluster'] = km.fit_predict(geo_features)

#Distance center
center_lon, center_lat = 2.3384444444444446, 48.86152777777778

df['dist_center'] =  np.sqrt((df['longitude'] - center_lon)**2 +
                             (df['latitude'] - center_lat) **2) * 111

In [5]:
metro_coords = np.radians(metro_df[['Latitude', 'Longitude']].values)
tree = BallTree(metro_coords, metric='haversine')

appart_coords = np.radians(df[['latitude', 'longitude']].values)

distances, indices = tree.query(appart_coords, k=1)
df['nearest_metro_dist_km'] = distances.flatten() * 6371

# Add nearest metro station info
df['nearest_metro_station'] = metro_df.iloc[indices.flatten()]['Libelle station'].values
df['nearest_metro_line'] = metro_df.iloc[indices.flatten()]['Libelle Line'].values

# Count metro stations within different radii
indices_300m = tree.query_radius(appart_coords, r=0.3/6371)
indices_500m = tree.query_radius(appart_coords, r=0.5/6371)

df['metro_count_300m'] = [len(idx) for idx in indices_300m]
df['metro_count_500m'] = [len(idx) for idx in indices_500m]
df['very_close_to_metro'] = (df['nearest_metro_dist_km'] < 0.1).astype(int)

In [6]:
df['surface_per_piece'] = df['surface_reelle_bati'] / df['nombre_pieces_principales'].replace(0, 1)
df['is_studio'] = (df['nombre_pieces_principales'] == 1).astype(int)
df['is_large'] = (df['nombre_pieces_principales'] >= 4).astype(int)

# Surface categories
df['surface_category'] = pd.cut(df['surface_reelle_bati'], 
                                  bins=[9, 40, 80, float('inf')],
                                  labels=['small', 'medium', 'large'])

In [7]:
# Sort by date
df_sorted = df.sort_values('date_mutation').reset_index(drop=True)

# Split: 80% train, 20% test
split_index = int(len(df_sorted) * 0.8)

train_df = df_sorted.iloc[:split_index].copy()
test_df = df_sorted.iloc[split_index:].copy()

In [8]:
station_stats_train = train_df.groupby('nearest_metro_station').agg({
    'surface_reelle_bati': ['mean', 'std', 'median', 'count'],
    'nombre_pieces_principales': ['mean', 'std', 'median'],
}).round(2)

# Flatten column names
station_stats_train.columns = [
    'station_avg_surface',
    'station_surface_std',
    'station_median_surface',
    'station_tx_count',
    'station_avg_rooms',
    'station_rooms_std',
    'station_median_rooms'
]

# Add derived features
station_stats_train['station_surface_range'] = (
    station_stats_train['station_avg_surface'] + 2*station_stats_train['station_surface_std'] -
    (station_stats_train['station_avg_surface'] - 2*station_stats_train['station_surface_std'])
)

# Replace zero std with 1 to avoid division by zero
station_stats_train['station_surface_std'] = station_stats_train['station_surface_std'].replace(0, 1)
station_stats_train['station_rooms_std'] = station_stats_train['station_rooms_std'].replace(0, 1)

In [9]:
# Merge to TRAIN (these are the same stations used to compute the stats)
train_df = train_df.merge(
    station_stats_train,
    left_on='nearest_metro_station',
    right_index=True,
    how='left'
)

# Merge to TEST (using the SAME stats from training!)
test_df = test_df.merge(
    station_stats_train,
    left_on='nearest_metro_station',
    right_index=True,
    how='left'
)

# Handle new stations in test that weren't in train
# Fill with overall training means
station_cols = [c for c in train_df.columns if c.startswith('station_')]

print(f"\nHandling missing values for new stations in test set...")
for col in station_cols:
    train_mean = train_df[col].mean()
    
    # Fill missing values in both train and test
    train_missing = train_df[col].isna().sum()
    test_missing = test_df[col].isna().sum()
    
    if train_missing > 0 or test_missing > 0:
        print(f"  {col}: filling {test_missing} missing values in test with train mean {train_mean:.2f}")
    
    train_df[col] = train_df[col].fillna(train_mean)
    test_df[col] = test_df[col].fillna(train_mean)


Handling missing values for new stations in test set...


In [10]:
for dataset_name, dataset in [('TRAIN', train_df), ('TEST', test_df)]:
    dataset['surface_vs_station_avg'] = (
        (dataset['surface_reelle_bati'] - dataset['station_avg_surface']) / 
        dataset['station_surface_std']
    )
    
    dataset['rooms_vs_station_avg'] = (
        (dataset['nombre_pieces_principales'] - dataset['station_avg_rooms']) / 
        dataset['station_rooms_std']
    )
    
    dataset['larger_than_station_median'] = (
        dataset['surface_reelle_bati'] > dataset['station_median_surface']
    ).astype(int)


In [11]:
train_agg = train_df.groupby(['code_postal', 'year', 'month']).agg({
    'price_per_sqrtm': 'median',
    'valeur_fonciere': 'median',
    'nombre_pieces_principales': 'median',
    'surface_reelle_bati': 'median',
    'surface_per_piece': 'median',
    'longitude': 'mean',
    'latitude': 'mean',
    'days_since_start': 'median',
    'geo_cluster': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'dist_center': 'median',
    'nearest_metro_dist_km': 'median',
    'station_tx_count': 'median',
    'station_avg_surface': 'median',
    'station_surface_std': 'median',
    'station_median_surface': 'median',
    'station_avg_rooms': 'median',
    'station_rooms_std': 'median',
    'station_median_rooms': 'median',
    'station_surface_range': 'median',
    'surface_vs_station_avg': 'median',
    'rooms_vs_station_avg': 'median',
    'larger_than_station_median': 'median',
    'metro_count_300m': 'median',
    'metro_count_500m': 'median',
    'very_close_to_metro': 'median',
    'is_studio': 'median',
    'is_large': 'median',
    'date_mutation': 'first'
}).reset_index()

print(f"Train aggregated shape: {train_agg.shape}")

Train aggregated shape: (837, 31)


In [12]:

test_agg = test_df.groupby(['code_postal', 'year', 'month']).agg({
    'price_per_sqrtm': 'median',
    'valeur_fonciere': 'median',
    'nombre_pieces_principales': 'median',
    'surface_reelle_bati': 'median',
    'surface_per_piece': 'median',
    'longitude': 'mean',
    'latitude': 'mean',
    'days_since_start': 'median',
    'geo_cluster': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'dist_center': 'median',
    'nearest_metro_dist_km': 'median',
    'station_tx_count': 'median',
    'station_avg_surface': 'median',
    'station_surface_std': 'median',
    'station_median_surface': 'median',
    'station_avg_rooms': 'median',
    'station_rooms_std': 'median',
    'station_median_rooms': 'median',
    'station_surface_range': 'median',
    'surface_vs_station_avg': 'median',
    'rooms_vs_station_avg': 'median',
    'larger_than_station_median': 'median',
    'metro_count_300m': 'median',
    'metro_count_500m': 'median',
    'very_close_to_metro': 'median',
    'is_studio': 'median',
    'is_large': 'median',
    'date_mutation': 'first'
}).reset_index()

In [13]:
# Transaction counts per postal code per month (computed separately)
train_tx_counts = train_df.groupby(['code_postal', 'year', 'month']).size().reset_index(name='transaction_count')
test_tx_counts = test_df.groupby(['code_postal', 'year', 'month']).size().reset_index(name='transaction_count')

train_agg = train_agg.merge(train_tx_counts, on=['code_postal', 'year', 'month'])
test_agg = test_agg.merge(test_tx_counts, on=['code_postal', 'year', 'month'])

# Historical activity - compute ONLY on training data
print("\nComputing historical transaction counts (TRAIN DATA ONLY)...")
historical_activity_train = train_df.groupby('code_postal').size().reset_index(name='total_transactions_train')

# Merge to both train and test (using TRAIN statistics)
train_agg = train_agg.merge(historical_activity_train, on='code_postal', how='left')
test_agg = test_agg.merge(historical_activity_train, on='code_postal', how='left')

# Handle postal codes in test that weren't in train
overall_avg_transactions = historical_activity_train['total_transactions_train'].mean()
test_agg['total_transactions_train'] = test_agg['total_transactions_train'].fillna(overall_avg_transactions)

# Market activity ratio (using training historical data)
train_agg['market_activity_ratio'] = train_agg['transaction_count'] / train_agg['total_transactions_train']
test_agg['market_activity_ratio'] = test_agg['transaction_count'] / test_agg['total_transactions_train']


Computing historical transaction counts (TRAIN DATA ONLY)...


In [14]:
# Define target and features
target = 'price_per_sqrtm'
drop_cols = ['price_per_sqrtm', 'valeur_fonciere', 'date_mutation']

# Split features and target
x_train = train_agg.drop(columns=drop_cols)
y_train = train_agg[target]

x_test = test_agg.drop(columns=drop_cols)
y_test = test_agg[target]

In [15]:
# Identify numeric and categorical features
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categoric_features = x_train.select_dtypes(include=['object', 'category']).columns.tolist()


numerical_pip = Pipeline([
    ("impute_num", SimpleImputer(strategy="median")),
    ("sc", StandardScaler())  
    ])
categoric_pip = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])


preprocessor = ColumnTransformer([
    ('cat', categoric_pip, categoric_features),
    ('num', numerical_pip, numeric_features)
])

In [16]:
#Fit and transform
x_train_processed = preprocessor.fit_transform(x_train)
x_test_processed = preprocessor.transform(x_test)

In [17]:
baseline_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=30,
    min_samples_leaf=1,
    min_samples_split=15,
    random_state=42,
    n_jobs=-1,
    criterion="absolute_error"
)

baseline_model.fit(x_train_processed, y_train)

# Predictions
y_train_pred = baseline_model.predict(x_train_processed)
y_test_pred = baseline_model.predict(x_test_processed)

# Evaluate
baseline_results = {
    'train_r2': r2_score(y_train, y_train_pred),
    'test_r2': r2_score(y_test, y_test_pred),
    'train_mae': mean_absolute_error(y_train, y_train_pred),
    'test_mae': mean_absolute_error(y_test, y_test_pred),
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred))
}

print("BASELINE MODEL RESULTS")
print(f"Train R²:  {baseline_results['train_r2']:.4f}")
print(f"Test R²:   {baseline_results['test_r2']:.4f}")
print(f"\nTrain MAE:  {baseline_results['train_mae']:.2f} €/m²")
print(f"Test MAE:   {baseline_results['test_mae']:.2f} €/m²")
print(f"\nTrain RMSE: {baseline_results['train_rmse']:.2f} €/m²")
print(f"Test RMSE:  {baseline_results['test_rmse']:.2f} €/m²")

BASELINE MODEL RESULTS
Train R²:  0.9645
Test R²:   0.9083

Train MAE:  205.32 €/m²
Test MAE:   353.10 €/m²

Train RMSE: 302.25 €/m²
Test RMSE:  508.44 €/m²


In [18]:
param_grid = {
    'n_estimators': [200, 250, 300],
    'max_depth': [25, 30, 35],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [15, 20, 25]
}



# Time-series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Grid search
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train_processed, y_train)

print("BEST PARAMETERS FOUND")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

print(f"\nBest CV MAE: {-grid_search.best_score_:.2f} €/m²")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
BEST PARAMETERS FOUND
max_depth: 35
min_samples_leaf: 2
min_samples_split: 25
n_estimators: 250

Best CV MAE: 1101.73 €/m²


In [19]:
optimized_model = grid_search.best_estimator_

# Predictions
y_train_pred_opt = optimized_model.predict(x_train_processed)
y_test_pred_opt = optimized_model.predict(x_test_processed)

# Evaluate
optimized_results = {
    'train_r2': r2_score(y_train, y_train_pred_opt),
    'test_r2': r2_score(y_test, y_test_pred_opt),
    'train_mae': mean_absolute_error(y_train, y_train_pred_opt),
    'test_mae': mean_absolute_error(y_test, y_test_pred_opt),
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred_opt)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred_opt))
}

print("OPTIMIZED MODEL RESULTS")
print(f"Train R²:  {optimized_results['train_r2']:.4f}")
print(f"Test R²:   {optimized_results['test_r2']:.4f}")
print(f"\nTrain MAE:  {optimized_results['train_mae']:.2f} €/m²")
print(f"Test MAE:   {optimized_results['test_mae']:.2f} €/m²")
print(f"\nTrain RMSE: {optimized_results['train_rmse']:.2f} €/m²")
print(f"Test RMSE:  {optimized_results['test_rmse']:.2f} €/m²")

OPTIMIZED MODEL RESULTS
Train R²:  0.9528
Test R²:   0.9052

Train MAE:  244.89 €/m²
Test MAE:   365.77 €/m²

Train RMSE: 348.54 €/m²
Test RMSE:  516.93 €/m²


In [20]:
#Vis
results_df = pd.DataFrame({
    'actual_prix_m2': y_test,
    'predicted_prix_m2': y_test_pred,
    'code_postal': x_test['code_postal'].values,
    'year': x_test['year'].values,
    'month': x_test['month'].values
})

#Vis pred vs actual
fig = px.scatter(results_df, x='actual_prix_m2', y='predicted_prix_m2',
                  hover_data=['code_postal', 'year', 'month'],
                  title='Predicted vs Actual Price per m²',
                  labels={
                       'predicted_prix_m2': 'Prix moyen €/m² prédit', 
                       'actual_prix_m2': 'Prix moyen €/m² réel'
                   })
fig.add_scatter(x=[y_test.min(), y_test.max()], 
                 y=[y_test.min(), y_test.max()],
                 mode='lines', name='Perfect Prediction', 
                 line=dict(dash='dash', color='red'))
fig.show()

In [21]:
#For Feature
feature_importance = pd.DataFrame({
    "feature_names": preprocessor.get_feature_names_out(),
    "coefficients": optimized_model.feature_importances_
})

In [22]:
#Vis Features
fig1 = px.bar(
    feature_importance.sort_values("coefficients", ascending=False),
    x="feature_names",
    y="coefficients",
    title="Random Forest Feature Importance for Price Prediction",
    labels={        
           'coefficients': 'Coefficients', 
           'feature_names': 'Features'
    }   
)

fig1.update_xaxes(
    tickvals=["num_code_postal", "num_longitude", "num_latitude", "num_surface_reelle_bati", "num_nombre_pieces_principales"],
    ticktext=["Code Postal", "Longitude", "Latitude", "Surface m²", "Nombre de pièces"]
)


fig1.show()

print(f"Feature importance not available for {feature_importance}")

Feature importance not available for                       feature_names  coefficients
0                  num__code_postal      0.340175
1    num__nombre_pieces_principales      0.000082
2          num__surface_reelle_bati      0.002904
3            num__surface_per_piece      0.002752
4                    num__longitude      0.029648
5                     num__latitude      0.054925
6             num__days_since_start      0.063492
7                  num__dist_center      0.003201
8        num__nearest_metro_dist_km      0.002915
9             num__station_tx_count      0.005573
10         num__station_avg_surface      0.013948
11         num__station_surface_std      0.047575
12      num__station_median_surface      0.001906
13           num__station_avg_rooms      0.006303
14           num__station_rooms_std      0.005009
15        num__station_median_rooms      0.000103
16       num__station_surface_range      0.049060
17      num__surface_vs_station_avg      0.002517
18        num

In [23]:
feature_importance_df = (
    feature_importance
    .sort_values("coefficients", ascending=False)
    .reset_index(drop=True)
)

feature_importance_df.head()

,feature_names,coefficients
0,num__total_transactions_train,0.360046
1,num__code_postal,0.340175
2,num__days_since_start,0.063492
3,num__latitude,0.054925
4,num__station_surface_range,0.049060


In [24]:
# Add this to the end of your notebook
import pickle

# Save model
with open("../src/model_random_forest.pkl", "wb") as f:
    pickle.dump(optimized_model, f)

# Save preprocessor
with open("../src/preprocessor_random_forest.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

In [25]:
import sklearn, pandas, numpy
print(sklearn.__version__, pandas.__version__, numpy.__version__)

1.6.1 2.2.3 2.1.3
